In [3]:
import polars as pl
from pathlib import Path

In [16]:
data_dir = Path("/s3-cache/aind-open-data")
f = data_dir / "HCR_749315_2025-05-08_14-00-00_processed_2025-05-17_22-15-31" / "HCR_749315_2025-05-08_14-00-00_processed_2025-05-17_22-15-31.parquet"

# Explore DF

In [17]:
df = pl.read_parquet(f)
df

chan,chan_spot_id,cell_id,round,z,y,x,z_center,y_center,x_center,dist,r,chan_488_bg,chan_488_fg,chan_514_bg,chan_514_fg,chan_561_bg,chan_561_fg,chan_594_bg,chan_594_fg,chan_638_bg,chan_638_fg,chan_488_intensity,chan_514_intensity,chan_561_intensity,chan_594_intensity,chan_638_intensity,over_thresh,unmixed_chan,dye_line_dist_ratio,valid_spot,unmixed_removed,spot_id
str,i64,i64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,str,f32,bool,bool,i64
"""488""",1,9,"""4""",74,990,5648,0.011567,-0.116883,0.253705,0.279574,0.7274562,152.0,284.6992,102.0,117.203255,160.0,184.08943,102.0,120.82927,112.04,127.5935,132.6992,15.203255,24.08943,18.82927,15.5535,true,"""488""",1.144353,false,false,0
"""488""",2,9,"""4""",77,1018,5694,-0.076886,-0.153813,0.030761,0.174689,0.608358,148.0,189.0,105.0,119.94309,163.0,182.30081,99.0,106.82114,115.0,129.95935,41.0,14.94309,19.30081,7.82114,14.95935,false,"""488""",1.080512,false,false,1
"""488""",3,9,"""4""",78,977,5657,-0.106324,0.243913,0.058419,0.272418,0.437502,164.0,229.53659,106.0,125.88618,181.04,249.20325,110.0,175.813,119.0,140.99187,65.53659,19.88618,68.16325,65.813,21.99187,true,null,null,null,true,2
"""488""",4,9,"""4""",78,1005,5653,-0.210846,-0.385967,-0.327366,0.548265,0.7405667,153.0,217.21138,103.0,112.84553,159.0,185.04878,98.0,108.0,116.0,128.33333,64.21138,9.84553,26.04878,10.0,12.33333,false,"""594""",1.040007,false,false,3
"""488""",5,9,"""4""",76,984,5653,-0.630952,-0.771316,1.0992422,1.4836987,0.58879,168.0,255.30081,107.0,132.03252,177.04,268.20325,114.0,204.34146,118.0,141.13008,87.30081,25.03252,91.16325,90.34146,23.13008,false,"""594""",6.089427,false,false,4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""638""",963741,44848,"""4""",1332,6055,4651,0.314389,0.38617,-0.318292,0.590997,0.8337266,101.0,115.48781,91.0,94.796745,103.0,130.61789,95.0,100.52846,89.0,116.76423,14.48781,3.796745,27.61789,5.52846,27.76423,true,"""561""",1.032534,false,false,8059233
"""638""",963742,44848,"""4""",1341,6034,4600,-0.133167,-0.312653,0.433892,0.5511329,0.748754,102.0,146.04065,91.0,99.69919,104.0,131.28455,98.0,118.796745,94.0,227.94308,44.04065,8.69919,27.28455,20.796745,133.94308,true,"""638""",2.406289,false,false,8059234
"""638""",963743,44848,"""4""",1337,6039,4667,0.015931,0.054527,0.3945396,0.398608,0.8690793,105.0,118.439026,93.0,98.13008,102.0,116.95935,97.0,105.95935,90.0,159.70732,13.439026,5.13008,14.95935,8.95935,69.70732,true,"""638""",3.170991,false,false,8059235


In [18]:
df["unmixed_removed"].value_counts()

unmixed_removed,count
bool,u32
false,5296771
true,2762467


## Investigate Sampling Bias

Let's check the distribution of `unmixed_removed` and see why sampling might miss the `True` values.

In [19]:
# Check the exact counts and percentages
total_rows = df.height
removed_true = df.filter(pl.col("unmixed_removed") == True).height
removed_false = df.filter(pl.col("unmixed_removed") == False).height

print(f"📊 Full Dataset Statistics:")
print(f"Total rows: {total_rows:,}")
print(f"unmixed_removed = True:  {removed_true:,} ({removed_true/total_rows*100:.3f}%)")
print(f"unmixed_removed = False: {removed_false:,} ({removed_false/total_rows*100:.3f}%)")

# Calculate probability of getting at least one True in a sample
sample_size = 5000
prob_all_false = (removed_false / total_rows) ** sample_size
prob_at_least_one_true = 1 - prob_all_false

print(f"\n🎲 Sampling Probability Analysis (sample size = {sample_size:,}):")
print(f"Probability all sampled are False: {prob_all_false:.6f}")
print(f"Probability at least one True: {prob_at_least_one_true:.6f}")

if prob_at_least_one_true < 0.95:
    print("⚠️ LOW PROBABILITY - You'll often miss True values in random samples!")
else:
    print("✅ GOOD PROBABILITY - Should usually see True values in samples")

📊 Full Dataset Statistics:
Total rows: 8,059,238
unmixed_removed = True:  2,762,467 (34.277%)
unmixed_removed = False: 5,296,771 (65.723%)

🎲 Sampling Probability Analysis (sample size = 5,000):
Probability all sampled are False: 0.000000
Probability at least one True: 1.000000
✅ GOOD PROBABILITY - Should usually see True values in samples


## Investigate NaN Filtering Issue

You're right! With 34% being `unmixed_removed = True`, they should appear in samples. Let's check if these rows have NaN values that might be getting filtered out.

In [20]:
# Let's check for NaN patterns by unmixed_removed status
print("🔍 Checking for NaN patterns that might cause filtering...")

# Get separate datasets
df_removed_true = df.filter(pl.col("unmixed_removed") == True)
df_removed_false = df.filter(pl.col("unmixed_removed") == False)

print(f"Dataset split:")
print(f"  unmixed_removed = True:  {df_removed_true.height:,} rows")
print(f"  unmixed_removed = False: {df_removed_false.height:,} rows")

# Check for null values in each group
print(f"\n📊 Null Value Analysis:")
all_cols = df.columns

for col in all_cols:
    nulls_true = df_removed_true[col].null_count()
    nulls_false = df_removed_false[col].null_count()
    
    total_true = df_removed_true.height
    total_false = df_removed_false.height
    
    pct_true = (nulls_true / total_true * 100) if total_true > 0 else 0
    pct_false = (nulls_false / total_false * 100) if total_false > 0 else 0
    
    # Only show columns with significant null differences
    if abs(pct_true - pct_false) > 5 or pct_true > 50 or pct_false > 50:
        print(f"  {col}:")
        print(f"    True group:  {nulls_true:,} nulls ({pct_true:.1f}%)")
        print(f"    False group: {nulls_false:,} nulls ({pct_false:.1f}%)")

🔍 Checking for NaN patterns that might cause filtering...
Dataset split:
  unmixed_removed = True:  2,762,467 rows
  unmixed_removed = False: 5,296,771 rows

📊 Null Value Analysis:
  unmixed_chan:
    True group:  2,762,467 nulls (100.0%)
    False group: 0 nulls (0.0%)
  dye_line_dist_ratio:
    True group:  2,762,467 nulls (100.0%)
    False group: 0 nulls (0.0%)
  valid_spot:
    True group:  2,762,467 nulls (100.0%)
    False group: 0 nulls (0.0%)


In [21]:
# Check if the API filtering might be the issue
# Let's simulate what the API does - check for required columns and missing values

# These are the required columns from the API
required_cols = ['spot_id', 'chan', 'r', 'dist', 'unmixed_chan', 'reassigned', 'unmixed_removed']

# Check intensity columns (these are dynamically determined)
intensity_cols = [col for col in df.columns if 'chan_' in col and '_intensity' in col]
print(f"🧪 Found intensity columns: {intensity_cols}")

all_needed_cols = required_cols + intensity_cols

print(f"\n🔍 Checking completeness for API required columns...")
print(f"Required columns: {required_cols}")
print(f"Intensity columns: {len(intensity_cols)} found")

# Check if rows with unmixed_removed=True have missing values in required columns
missing_by_group = {}

for col in all_needed_cols:
    if col in df.columns:
        nulls_true = df_removed_true[col].null_count()
        nulls_false = df_removed_false[col].null_count()
        
        if nulls_true > 0 or nulls_false > 0:
            missing_by_group[col] = {
                'true_nulls': nulls_true,
                'false_nulls': nulls_false,
                'true_pct': nulls_true / df_removed_true.height * 100,
                'false_pct': nulls_false / df_removed_false.height * 100
            }

if missing_by_group:
    print(f"\n⚠️ Columns with missing values in required fields:")
    for col, stats in missing_by_group.items():
        print(f"  {col}:")
        print(f"    True group:  {stats['true_nulls']:,} nulls ({stats['true_pct']:.1f}%)")
        print(f"    False group: {stats['false_nulls']:,} nulls ({stats['false_pct']:.1f}%)")
        
        if stats['true_pct'] > stats['false_pct'] + 10:
            print(f"    🚨 TRUE GROUP HAS MUCH MORE MISSING DATA!")
else:
    print("✅ No missing values found in required columns")

🧪 Found intensity columns: ['chan_488_intensity', 'chan_514_intensity', 'chan_561_intensity', 'chan_594_intensity', 'chan_638_intensity']

🔍 Checking completeness for API required columns...
Required columns: ['spot_id', 'chan', 'r', 'dist', 'unmixed_chan', 'reassigned', 'unmixed_removed']
Intensity columns: 5 found

⚠️ Columns with missing values in required fields:
  unmixed_chan:
    True group:  2,762,467 nulls (100.0%)
    False group: 0 nulls (0.0%)
    🚨 TRUE GROUP HAS MUCH MORE MISSING DATA!


In [22]:
# Let's check the valid_spot filter that the API applies
print(f"🎯 Checking valid_spot filtering (this is applied by the API):")
print(f"valid_spot value counts:")
print(df["valid_spot"].value_counts())

# Check valid_spot by unmixed_removed status
valid_by_removed = df.group_by(["unmixed_removed", "valid_spot"]).agg(pl.len().alias("count"))
print(f"\n📊 Cross-tabulation: unmixed_removed vs valid_spot")
print(valid_by_removed.sort(["unmixed_removed", "valid_spot"]))

# Calculate the impact
valid_true_group = df.filter((pl.col("unmixed_removed") == True) & (pl.col("valid_spot") == True)).height
valid_false_group = df.filter((pl.col("unmixed_removed") == False) & (pl.col("valid_spot") == True)).height

total_valid = valid_true_group + valid_false_group

print(f"\n🧮 After valid_spot filtering:")
print(f"  unmixed_removed = True & valid_spot = True:  {valid_true_group:,}")
print(f"  unmixed_removed = False & valid_spot = True: {valid_false_group:,}")
print(f"  Total valid spots: {total_valid:,}")

if total_valid > 0:
    pct_true_in_valid = valid_true_group / total_valid * 100
    print(f"  Percentage of valid spots that are unmixed_removed = True: {pct_true_in_valid:.2f}%")
    
    if pct_true_in_valid < 1:
        print(f"  🚨 FOUND THE ISSUE! Only {pct_true_in_valid:.2f}% of valid spots have unmixed_removed = True")
        print(f"  This explains why you rarely see them in samples!")
    else:
        print(f"  ✅ Should see unmixed_removed = True in samples with {pct_true_in_valid:.1f}% probability")
else:
    print("  ⚠️ No valid spots found!")

🎯 Checking valid_spot filtering (this is applied by the API):
valid_spot value counts:
shape: (3, 2)
┌────────────┬─────────┐
│ valid_spot ┆ count   │
│ ---        ┆ ---     │
│ bool       ┆ u32     │
╞════════════╪═════════╡
│ null       ┆ 2762467 │
│ true       ┆ 1742904 │
│ false      ┆ 3553867 │
└────────────┴─────────┘

📊 Cross-tabulation: unmixed_removed vs valid_spot
shape: (3, 3)
┌─────────────────┬────────────┬─────────┐
│ unmixed_removed ┆ valid_spot ┆ count   │
│ ---             ┆ ---        ┆ ---     │
│ bool            ┆ bool       ┆ u32     │
╞═════════════════╪════════════╪═════════╡
│ false           ┆ false      ┆ 3553867 │
│ false           ┆ true       ┆ 1742904 │
│ true            ┆ null       ┆ 2762467 │
└─────────────────┴────────────┴─────────┘

🧮 After valid_spot filtering:
  unmixed_removed = True & valid_spot = True:  0
  unmixed_removed = False & valid_spot = True: 1,742,904
  Total valid spots: 1,742,904
  Percentage of valid spots that are unmixed_removed =

## Test Sampling with Different Filters

Let's test sampling with and without the `valid_spot` filter to confirm this is the issue:

In [ ]:
# Test 1: Sample from full dataset (no filters)
print("🎲 Test 1: Sampling from FULL dataset (no filters)")
sample_full = df.sample(n=5000, seed=42)
sample_full_counts = sample_full["unmixed_removed"].value_counts()
print("Sample results:")
print(sample_full_counts)

# Test 2: Sample from valid_spot filtered dataset (like API does)
print(f"\n🎲 Test 2: Sampling from VALID_SPOT filtered dataset (like API)")
df_valid = df.filter(pl.col("valid_spot") == True)
if df_valid.height > 0:
    sample_size = min(5000, df_valid.height)
    sample_valid = df_valid.sample(n=sample_size, seed=42)
    sample_valid_counts = sample_valid["unmixed_removed"].value_counts()
    print("Sample results:")
    print(sample_valid_counts)
else:
    print("No valid spots found!")

# Test 3: Sample ensuring we get both types (stratified sampling)
print(f"\n🎲 Test 3: STRATIFIED sampling (force both types)")
if df_valid.height > 0:
    # Get some of each type
    df_valid_true = df_valid.filter(pl.col("unmixed_removed") == True)
    df_valid_false = df_valid.filter(pl.col("unmixed_removed") == False)
    
    # Sample proportionally or at least get some of each
    if df_valid_true.height > 0 and df_valid_false.height > 0:
        n_true = min(1000, df_valid_true.height)  # At least 1000 or all available
        n_false = min(4000, df_valid_false.height)  # Rest false
        
        sample_true = df_valid_true.sample(n=n_true, seed=42)
        sample_false = df_valid_false.sample(n=n_false, seed=42)
        
        sample_stratified = pl.concat([sample_true, sample_false])
        sample_stratified_counts = sample_stratified["unmixed_removed"].value_counts()
        print("Stratified sample results:")
        print(sample_stratified_counts)
    else:
        print("Cannot create stratified sample - one group is empty")
else:
    print("No valid spots for stratified sampling!")

# API requests

In [ ]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Any

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

In [ ]:
# API Configuration
BASE_URL = "http://localhost:9999"  # Adjust port if different
API_ENDPOINT = f"{BASE_URL}/api/real_spots_data"

# Request parameters
params = {
    "sample_size": 5000,        # Number of spots to sample
    "force_refresh": False      # Set to True to bypass cache
}

print(f"API Endpoint: {API_ENDPOINT}")
print(f"Parameters: {params}")